In [2]:
#from datetime import datetime
#!pip install elasticsearch --user
from elasticsearch import Elasticsearch
#!pip install python-greeklish --user
from greeklish.converter import Converter
import pandas as pd

In [3]:
# read the dataset that is created every day
#df_products = pd.read_pickle('/home/ubuntu/Spitishop/Spitishop_RE/search_products_api.pkl')
df_products = pd.read_pickle("data/df_products_new.pkl")

products_search = df_products.copy()

In [4]:
# add a column for the exact match on 'Product_name_eng' column
#df_products['Product_name_eng_exact'] = df_products['Product_name_eng'].copy()

In [5]:
df_products.head(5)

,Code,Price,Product_name,Product_name_eng,name,freq,categories,categories_text,categories_text_eng,attributes,attributes_text,attributes_text_eng,brand_name,reference
0,29415,85.000000,Πάπλωμα Κούνιας Πουπουλένιο Guy Laroche Bebe 100%,paploma kounias poupoulenio guy laroche bebe 100%,Βρεφικά,8.0,"[Βρεφικά, Παπλώματα Βρεφικά]",Βρεφικά Παπλώματα Βρεφικά,brefika paplwmata brefika,NaN,Guy Laroche,guy laroche,Guy Laroche,000000700022
2,29609,10.000000,Διακοσμητική Μαξιλαροθήκη (40x40) Guy Laroche ...,diakosmhtikh maxilarothhkh (40x40) guy laroche...,Σαλόνι,4.0,"[Σαλόνι, Διακοσμητικά Μαξιλάρια]",Σαλόνι Διακοσμητικά Μαξιλάρια,saloni diakosmhtika maxilaria,NaN,Guy Laroche,guy laroche,Guy Laroche,1116030113008
4,29611,10.000000,Διακοσμητική Μαξιλαροθήκη (40x40) Guy Laroche ...,diakosmhtikh maxilarothhkh (40x40) guy laroche...,Σαλόνι,4.0,"[Σαλόνι, Διακοσμητικά Μαξιλάρια]",Σαλόνι Διακοσμητικά Μαξιλάρια,saloni diakosmhtika maxilaria,NaN,Guy Laroche,guy laroche,Guy Laroche,1116030113010
6,29675,21.999999,Ζεύγος Καλύμματα Μαξιλαριών Αδιάβροχα Guy Laro...,zeugos kalymmata maxilariwn adiabroxa guy laro...,Κρεβατοκάμαρα,8.0,"[Κρεβατοκάμαρα, Παιδικά, Επιστρώματα, Καλύμματ...",Κρεβατοκάμαρα Παιδικά Επιστρώματα Καλύμματα Μα...,krebatokamara paidika epistrwmata kalymmata ma...,NaN,Guy Laroche,guy laroche,Guy Laroche,1118086212005
8,29865,59.000000,Μπουρνούζι Guy Laroche Linda Mellon,mpournouzi guy laroche linda mellon,Μπάνιο,4.0,"[Μπάνιο, Μπουρνούζια, Δώρα γάμου, Για εκείνη]",Μπάνιο Μπουρνούζια Δώρα γάμου Για εκείνη,mpanio mpournouzia dwra gamou gia ekeinh,[SMALL],Guy Laroche SMALL,guy laroche small,Guy Laroche,G.1300616


## -- Create a new elastic search index with the new dataset

In [8]:
# create a list of dicts with the product details
products = []
tmp = df_products.apply(lambda x: products.append({'title_eng':x['Product_name_eng'],
                                                   #'title_eng_exact':x['Product_name_eng_exact'],
                                                         'title_init':x['Product_name'],
                                                         #'url':x['Url'],
                                                         'Price':x['Price'],
                                                         #'Brand':x['Brand'],
                                                         'Category':x['name'],
                                                         #'img_url':x['img_url'],
                                                         'Code':x['Code'],
                                                         'Popularity':x['freq'],
                                                         'categ_eng':x['categories_text_eng'],
                                                         'attr_eng':x['attributes_text_eng'],
                                                         'reference':x['reference']
                                                  }), axis=1)

In [6]:
# connect to elasticsearch / create an Elasticsearch object
def connect_elasticsearch():
    _es = None
    #_es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    _es = Elasticsearch()
    if _es.ping():
        print('- Connected')
    else:
        print('- Problem with the connection')
    return _es

es = connect_elasticsearch()

- Connected


In [7]:
# # delete the old index
#es.indices.delete(index='spitishop_testing', ignore=[400, 404])

{'acknowledged': True}

In [9]:
def create_index(es_object, index_name='spitishop_testing'):
    
    # index settings
#     settings = {
#           "settings": {
#             "analysis": {
#               "analyzer": {
#                 "my_analyzer": { 
#                   "type": "standard", # https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-standard-analyzer.html
#                   "stopwords": ["sentoni", "the", 'mpournouzi'],
#                   #"stopwords_path": "stopwords/my_stopwords.txt"
#                 }
#               }
#             }
#           },
#         #"mappings": {}
#     }

    # https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-synonym-tokenfilter.html
    settings = {
                "settings": {
                    "index" : {
                        "analysis" : {
                            "analyzer" : {
                                "my_analyzer" : {
                                    "tokenizer" : "standard",
                                    "filter" : ["my_stop", "synonym"]
                                }
                            },
                            "filter" : {
                                "my_stop": {
                                    "type" : "stop",
                                    "stopwords": ["to", "kai"]
                                },
                                "synonym" : {
                                    "type" : "synonym",
                                    "lenient": True,
                                    "synonyms" : ["ypnodomatio, bedroom => krebatokamara"]
                                }
                            }
                        }
                    }
                },
#               "mappings": {
#                     "properties": {
#                       "title_eng_exact":{"type":"string", "index":"not_analyzed"},    
#                     }
#                   }
            }
    
    try:
        #if not es_object.indices.exists(index_name):
        # Ignore 400 means to ignore "Index Already Exist" error.
        es_object.indices.create(index=index_name, ignore=400, body=settings)
        print('Created Index')
    except Exception as ex:
        print('Error: ', str(ex))

create_index(es, 'spitishop_testing')

# add products to the index
i = 1
for product in products[0:]:
    es.index(index='spitishop_testing',
             doc_type='product',
             id=i,
             body=product,
            )
    i += 1

Created Index


In [10]:
def get_results_spitishop(query, category, exact):

    # first check the case that the query is just a reference id of a specific number
    df_product = products_search[products_search.reference==query]
    if len(df_product)>0:
        #print('-- The product code was found')
        df_product = df_product[['Code', 'freq']]
        df_product['score'] = 1
        df_product.columns = ['Code', 'Popularity', 'score']
        results = df_product.to_dict('records')

        return results, 'elastic search'   
    
    # convert the query to lowercase and some letters based on greek_letters_dict
    greek_letters_dict = {'ω':'ο'}
    query = query.lower().translate(str.maketrans(greek_letters_dict))
    # convert it to greeklish
    # create an object for greeklish translation
    conv = Converter(max_expansions=1)
    query = conv.convert(query)[0]
    
    # search_flag defines whether the results are procuced by elasticsearch or based on popularity
    search_flag = 0 
    
    # search the index based on the query    
    if category=='All':
        if exact == True:
            print("Exact match")
            # search the index based on the query
            results = es.search(index='spitishop_testing', body = { "from" : 0, "size" : 1000,
#                 'query':{
#                     "wildcard": {
#                        "title_eng_exact": {
#                           "value": "*"+query+"*",
#                        }
#                     }
#                 }
                                                                   
#                     "query": {
#                         "match_phrase": { # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html
#                           "title_eng": {
#                             "query": query,
#                           }
#                         }
#                       }
                 
                      "query": {
                        "match": {
                          "title_eng": {
                            "query": query,
                            "operator":"AND"
                          }
                        }
                      }
                                                                  })
                                                                   
            
            # if no results are found, return nothing
            if len(results['hits']['hits'][0:])==0:
                return 'No match'
            
        else:    
            # search the index based on the query
            # search the query inside 'title_eng', 'categ_eng' & 'attr_eng' and give different boost in each field
            # https://www.elastic.co/guide/en/elasticsearch/guide/current/multi-query-strings.html
            results = es.search(index='spitishop_testing', body = { "from" : 0, "size" : 1000,
#                 'query':{
#                     'bool': {
#                       'must': [
#                                     {'match':{'title_eng':{"query" : query,
#                                                            "fuzziness": "AUTO:3,5", # https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#fuzziness
#                                                            #"fuzziness": 2,
#                                                            #"fuzzy_transpositions":False,
#                                                             "analyzer": "my_analyzer"

#                                                           }}},
#                                     #{'match': {'Category': category}},
#                                ],

#                     }
#                 }
              "query": {
                "bool": {
                  "should": [
                    { "match": { 
                        "title_eng":  {
                          "query": query,
                          "fuzziness": "AUTO:3,5",
                          "analyzer": "my_analyzer",  
                          "boost": 9
                    }}},
                    { "match": { 
                        "categ_eng":  {
                          "query": query,
                          "fuzziness": "AUTO:3,5",
                          "analyzer": "my_analyzer",                             
                          "boost": 2
                    }}},
                    { "match": { 
                        "attr_eng":  {
                          "query": query,
                          "fuzziness": "AUTO:3,5",
                          "analyzer": "my_analyzer",                            
                          "boost": 1
                    }}},
                  ]
                }
              }                                                                   
            },                  
                               )
    else:
        results = es.search(index='spitishop_testing', body = { "from" : 0, "size" : 1000,
            'query':{
                'bool': {
                  'must': [
                                {'match':{'title_eng':{"query" : query,
                                                       "fuzziness": "AUTO:3,5", # https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#fuzziness
                                                       #"fuzziness": 2,
                                                       #"fuzzy_transpositions":False,
                                                        "analyzer": "my_analyzer"

                                                      }}},
                                {'match': {'Category': category}},
                           ],
#                   "filter": [ 
#                                 {"term":{"Category":category}},
#                             ]
                    
                }
            }
        },                  
                           )

    # list of results
    res = results['hits']['hits'][0:]
    
    # if the search query doesnt have results in the specified category, then check in all categories
    if (len(res)==0) & (category!='All'):
        print(' - Searching regardless the specified category')
        results = es.search(index='spitishop_1', body = { "from" : 0, "size" : 1000,
            'query':{
                'bool': {
                  'must': [
                                {'match':{'title_eng':{"query" : query,
                                                       "fuzziness": "AUTO:3,5", # https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#fuzziness
                                                       #"fuzziness": 2,
                                                       #"fuzzy_transpositions":False,
                                                        "analyzer": "my_analyzer"

                                                      }}},
                                #{'match': {'Category': category}},
                           ],
#                   "filter": [ 
#                                 {"term":{"Category":category}},
#                             ]
                    
                }
            }
        },                  
                           )
        res = results['hits']['hits'][0:]
        
    results = []
    for i in range(len(res)):
        #print(res[i]['_source']['title_init']+' - ', res[i]['_source']['Price'])
        result = res[i]

        results.append({
            'Code':result['_source']['Code'],
            'Product_name':result['_source']['title_init'],
                       'title_eng':result['_source']['title_eng'],
                        #'Category':result['_source']['Category'],
                        #'Price':result['_source']['Price'],
                        'score':result['_score'],
                        #'img_url':result['_source']['img_url'],
                        #'Url':result['_source']['url'],
                        'Popularity':result['_source']['Popularity'],
                        'categories':result['_source']['categ_eng'],
                        'attributes':result['_source']['attr_eng'],
                       })

    df_results = pd.DataFrame(results)

    if len(df_results) > 0:
        # convert list of dicts to pandas dataframe 
        #df_results = pd.DataFrame(results)

        # keep only results with relevance score higher that 4
        #df_results = df_results[df_results.score>4]

        # sort results based on relevance and Price/popularity
        df_results = df_results.sort_values(by=['score', 'Popularity'], ascending=[False, False])

        # search_flag defines whether the results are procuced by elasticsearch or based on popularity
        search_flag = 'elastic search'

    if len(df_results) == 0:
        #print('- No results were returned via elasticsearch. So, return popular products.')
        df_results = products_search.sort_values(by=['freq'], ascending=[False]).head(60)
        df_results = df_results[['Code', 'freq']]
        df_results['score'] = 0
        df_results.columns = ['Code', 'Popularity', 'score']

        # set the flag==1 - the resutls are produced based on popularity
        search_flag = 'popularity'
        
    results = df_results.to_dict('records')
    
#     results2 = []
#     for i in range(200):
#         results2.append(results[i*5:5+(i*5)])
    
    return results, search_flag

In [14]:
query = '5205495445618'
category = 'All'
exact = False
get_results_spitishop(query, category, exact)

([{'Code': 120401.0, 'Popularity': 20.0, 'score': 1.0}], 'elastic search')

In [46]:
#!sudo cp /home/ubuntu/Spitishop/Spitishop_RE/search_products_api_tmp.pkl /home/spitishop/spitishop_rec_app

In [16]:
#!/opt/anaconda3/bin/ipython /home/ubuntu/Spitishop/Spitishop_RE/products_search.py